In [1]:
import tensorflow as tf
import numpy as np

In [2]:
print(tf.__version__)

2.6.0


In [3]:
import pandas as pd
import pickle

In [4]:
import pickle
with open('/mnt/datasets/ml/ml-1m/all_ratings.pkl', 'rb') as f:    
    user_dict = pickle.load(f)
items_num=3416
maxlen=50
len_seq=10
batch_size=512
epoch_num=20
hidden_size=64
keep_rate=0.9
layers_num=2
num_interest=1
neg_num=4
test_neg_num=1000

In [5]:
import random
import numpy as np
def sample(user_dict,maxlen,len_seq):
    train_set=[]
    train_val_set=[]
    val_set=[]
    test_set=[]
    for u in user_dict.keys():
        idx=0
        hist=user_dict[u]        
        hist=hist[-maxlen:-2]
        #print(hist)
        for i in range(1,len(hist)):
            seq = np.zeros([len_seq], dtype=np.int32)
            #print(hist[0:i])
            seq[max(0,len_seq+idx-1):]=hist[max(0,i-len_seq):i]
            idx+=-1
            nxt = hist[i]
            #print((u,seq,nxt))
            train_set.append((u,list(seq),nxt))
            #print(seq)
        train_val_set.append((u,list(seq),nxt))
        seq = np.zeros([len_seq], dtype=np.int32)
        seq=hist[-len_seq-2:-2]
        nxt = user_dict[u][-2]
        val_set.append((u,list(seq),nxt))
        seq = np.zeros([len_seq], dtype=np.int32)
        seq=hist[-len_seq-1:-1]
        nxt = user_dict[u][-1]
        test_set.append((u,list(seq),nxt))
    random.shuffle(train_set)
    random.shuffle(test_set)
    return train_set,test_set,val_set,train_val_set
            
def non_zero_sample(user_dict,maxlen,len_seq):
    train_set=[]
    test_set=[]
    val_set=[]
    train_val_set=[]
    for u in user_dict.keys():
        idx=0
        hist=user_dict[u]        
        seq = np.zeros([len_seq], dtype=np.int32)
        #print(np.shape(seq[-maxlen:]))
        while(len(hist)<maxlen):
            hist.insert(0,0)
        #print(len(hist),hist)
        hist=hist[-maxlen:-2]
        for i in range(len_seq,len(hist)):
            seq = np.zeros([len_seq], dtype=np.int32)
            #rint(hist[max(0,i-len_seq):i])
            seq=hist[max(0,i-len_seq):i]
            idx+=-1
            nxt = hist[i]
            #print((u,seq,nxt))
            neg_item = [random.randint(1, items_num) for _ in range(neg_num)]
            train_set.append((u,list(seq),nxt,neg_item))
            #print(seq)
        #print(np.shape(hist[0:len(hist)-1]))
        train_val_set.append((u,list(seq),nxt))
        seq = np.zeros([len_seq], dtype=np.int32)
        seq=hist[-len_seq-2:-2]
        nxt = user_dict[u][-2]
        neg_item = [random.randint(1, items_num) for _ in range(neg_num)]
        val_set.append((u,list(seq),nxt,neg_item))
        seq = np.zeros([len_seq], dtype=np.int32)
        seq=hist[-len_seq-1:-1]
        nxt = user_dict[u][-1]
        neg_item = [random.randint(1, items_num) for _ in range(test_neg_num)]
        test_set.append((u,list(seq),nxt,neg_item))
    random.shuffle(train_set)
    random.shuffle(test_set)
    return train_set,test_set ,val_set,train_val_set   

In [6]:
train_set,test_set,val_set,train_val_set=non_zero_sample(user_dict,maxlen,len_seq)
print(len(train_set),len(test_set),len(val_set),len(train_val_set))
users_num=len(test_set)

229520 6040 6040 6040


In [7]:
users, click_seqs, pos_items,neg_items = zip(*train_set)
train_data = {'click_seq': np.array(click_seqs), 'pos_item': np.array(pos_items),'neg_item':np.array(neg_items)}
users, click_seqs, pos_items,neg_items = zip(*test_set)
test_data = {'click_seq': np.array(click_seqs), 'pos_item': np.array(pos_items),'neg_item':np.array(neg_items)}

In [8]:
print(np.shape(train_data['neg_item']))

(229520, 4)


In [9]:
class mlp_rec(tf.keras.models.Model):
    def __init__(self):
        super(mlp_rec, self).__init__()
        self.len_seq=len_seq
        self.embedding = tf.keras.layers.Embedding(items_num+1,hidden_size,input_length=len_seq)
        self.w = tf.Variable(initial_value= tf.random.normal([len_seq,1]),name='w')
        self.b = tf.Variable(initial_value= tf.random.normal([1]),name='b')
    
    def call(self, inputs):
        seq_emb=self.embedding(inputs['click_seq'])
        seq_emb=tf.reshape(seq_emb,[tf.shape(seq_emb)[0],hidden_size,len_seq])
        seq_emb = tf.reshape(tf.matmul(seq_emb,self.w),[tf.shape(seq_emb)[0],1,hidden_size])+self.b
        seq_emb=tf.squeeze(seq_emb,axis=1)
        
        pos_emb=self.embedding(tf.reshape(inputs['pos_item'], [-1, ]))
        neg_emb=self.embedding(inputs['neg_item'])
        
        pos_score=tf.reshape(tf.reduce_sum(tf.multiply(seq_emb,pos_emb),axis=1),[-1,1])
        neg_score=tf.reduce_sum(tf.multiply(tf.expand_dims(seq_emb,1),neg_emb),axis=-1)
        pos_scores = tf.tile(pos_score, [1, neg_score.shape[1]])
        loss = tf.reduce_mean(- tf.math.log(tf.nn.sigmoid(pos_score)) - tf.math.log(1 - tf.nn.sigmoid(neg_score))) / 2
        self.add_loss(loss)
        logits = tf.concat([pos_score, neg_score], axis=-1)
        return logits
    
    def summary(self):
        inputs = {
            'click_seq': tf.keras.layers.Input(shape=(self.len_seq,), dtype=tf.int32),
            'pos_item': tf.keras.layers.Input(shape=(), dtype=tf.int32),
            'neg_item': tf.keras.layers.Input(shape=(1,), dtype=tf.int32)  # suppose neg_num=1
        }
        tf.keras.models.Model(inputs=inputs, outputs=self.call(inputs)).summary()

In [10]:
tf.keras.backend.clear_session()

model = mlp_rec()
#model.summary()
optimizer=tf.keras.optimizers.Nadam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004)
model.compile(optimizer=optimizer)

2022-10-18 11:53:20.978420: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-18 11:53:21.526841: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10801 MB memory:  -> device: 0, name: Tesla K80, pci bus id: 0000:0a:00.0, compute capability: 3.7


In [11]:
def hr(rank, k):
    """Hit Rate.
    Args:
        :param rank: A list.
        :param k: A scalar(int).
    :return: hit rate.
    """
    res = 0.0
    for r in rank:
        if r < k:
            res += 1
    return res / len(rank)


def mrr(rank, k):
    """Mean Reciprocal Rank.
    Args:
        :param rank: A list.
        :param k: A scalar(int).
    :return: mrr.
    """
    mrr = 0.0
    for r in rank:
        if r < k:
            mrr += 1 / (r + 1)
    return mrr / len(rank)


def ndcg(rank, k):
    """Normalized Discounted Cumulative Gain.
    Args:
        :param rank: A list.
        :param k: A scalar(int).
    :return: ndcg.
    """
    res = 0.0
    for r in rank:
        if r < k:
            res += 1 / np.log2(r + 2)
    return res / len(rank)


def eval_rank(pred_y, metric_names, k=10):
    """Evaluate
        Args:
            :param pred_y: A ndarray.
            :param metric_names: A list like ['hr'].
            :param k: A scalar(int).
        :return: A result dict such as {'hr':, 'ndcg':, ...}
    """
    rank = pred_y.argsort().argsort()[:, 0]
    res_dict = {}
    for name in metric_names:
        if name == 'hr':
            res = hr(rank, k)
        elif name == 'ndcg':
            res = ndcg(rank, k)
        elif name == 'mrr':
            res = mrr(rank, k)
        else:
            break
        res_dict[name] = res
    return res_dict

def eval_pos_neg(model, test_data, metric_names, k=10, batch_size=None):
    """Evaluate the performance of Top-k recommendation algorithm.
    Note: Test data must contain some negative samples(>= k) and one positive samples.
    Args:
        :param model: A model built-by tensorflow.
        :param test_data: A dict.
        :param metric_names: A list like ['hr'].
        :param k: A scalar(int).
        :param batch_size: A scalar(int).
    :return: A result dict such as {'hr':, 'ndcg':, ...}
    """
    pred_y = - model.predict(test_data, batch_size)
    return eval_rank(pred_y, metric_names, k)

In [12]:
from time import time
for epoch in range(1, epoch_num + 1):
    t1 = time()
    model.fit(
            x=train_data,
            epochs=1,
            batch_size=batch_size)
    t2 = time()
    eval_dict = eval_pos_neg(model, test_data, ['hr', 'mrr', 'ndcg'], 10, batch_size)
    print('Iteration %d Fit [%.1f s], Evaluate [%.1f s]: HR = %.4f, MRR = %.4f, NDCG = %.4f'
              % (epoch, t2 - t1, time() - t2, eval_dict['hr'], eval_dict['mrr'], eval_dict['ndcg']))

2022-10-18 11:53:21.803361: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


449/449 [==============================] - 4s 5ms/step - loss: 0.5769
Iteration 1 Fit [3.6 s], Evaluate [1.0 s]: HR = 0.0106, MRR = 0.0028, NDCG = 0.0046
449/449 [==============================] - 2s 5ms/step - loss: 0.5441
Iteration 2 Fit [2.5 s], Evaluate [0.9 s]: HR = 0.0129, MRR = 0.0040, NDCG = 0.0060
449/449 [==============================] - 3s 6ms/step - loss: 0.4320
Iteration 3 Fit [5.2 s], Evaluate [1.0 s]: HR = 0.0189, MRR = 0.0058, NDCG = 0.0088
449/449 [==============================] - 2s 5ms/step - loss: 0.2920
Iteration 4 Fit [2.6 s], Evaluate [0.9 s]: HR = 0.0154, MRR = 0.0054, NDCG = 0.0077
449/449 [==============================] - 2s 5ms/step - loss: 0.2222
Iteration 5 Fit [2.6 s], Evaluate [0.9 s]: HR = 0.0149, MRR = 0.0046, NDCG = 0.0069
449/449 [==============================] - 2s 5ms/step - loss: 0.1863
Iteration 6 Fit [2.6 s], Evaluate [0.9 s]: HR = 0.0174, MRR = 0.0050, NDCG = 0.0078
449/449 [==============================] - 2s 5ms/step - loss: 0.1623
Iterat

In [13]:
a=tf.constant([[2558, 2559, 2582, 2605, 2608, 2669, 2801, 2835, 2913, 3052]])
tf.print(tf.argmax(model(a)[0]))

TypeError: Only integers, slices (`:`), ellipsis (`...`), tf.newaxis (`None`) and scalar tf.int32/tf.int64 tensors are valid indices, got 'click_seq'

In [ ]:
'''def train_model(model,ds_train,ds_valid,epoches):

    for epoch in tf.range(1,epoches+1):
        model.reset_metrics()
        
        # 在后期降低学习率
        if epoch == 5000:
            model.optimizer.lr.assign(model.optimizer.lr/2.0)
            tf.print("Lowering optimizer Learning Rate...\n\n")
        
        for x, y in ds_train:
            train_result = model.train_on_batch(x, y)

        for x, y in ds_valid:
            valid_result = model.test_on_batch(x, y,reset_metrics=False)
            
        if epoch%100 ==0:
            tf.print("epoch = ",epoch)
            print("train:",dict(zip(model.metrics_names,train_result)))
            print("valid:",dict(zip(model.metrics_names,valid_result)))
            print("")'''

In [ ]:
'''train_model(model,ds_train,ds_test,800)'''